## Benchmarking Perceptron


#### About profiling julia code

- https://thirld.com/blog/2015/05/30/julia-profiling-cheat-sheet/

#### Examples of speeding up code

There is a small number of "tricks" that can be applied to speed up execution time and save memory allocations. This is paramount for enjoying C like speed with julia code.

- https://discourse.julialang.org/t/speed-up-this-code-game/3666

In [1]:
peakflops()

8.945040135975792e10

In [2]:
versioninfo()

Julia Version 0.6.0-rc2.0
Commit 68e911be53 (2017-05-18 02:31 UTC)
Platform Info:
  OS: macOS (x86_64-apple-darwin13.4.0)
  CPU: Intel(R) Xeon(R) CPU E5-1620 v2 @ 3.70GHz
  WORD_SIZE: 64
  BLAS: libopenblas (USE64BITINT DYNAMIC_ARCH NO_AFFINITY Sandybridge)
  LAPACK: libopenblas64_
  LIBM: libopenlibm
  LLVM: libLLVM-3.9.1 (ORCJIT, ivybridge)


In [3]:
using MNIST
using BenchmarkTools

source_path = join(push!(split(pwd(),"/")[1:end-1],"source/" ),"/")

if !contains(==,LOAD_PATH, source_path) 
    push!(LOAD_PATH, source_path)
end

using MulticlassPerceptron4
using MulticlassPerceptron3
using MulticlassPerceptron2
using MulticlassPerceptron1

percep1 = MulticlassPerceptron1.MPerceptron(Float32, 10, 784)
percep2 = MulticlassPerceptron2.MPerceptron(Float32, 10, 784)
percep3 = MulticlassPerceptron3.MPerceptron(Float32, 10, 784)
percep4 = MulticlassPerceptron4.MPerceptron(Float32, 10, 784)

n_classes = 10
n_features = 784

784

In [4]:
X_train, y_train = MNIST.traindata();
X_test, y_test = MNIST.testdata();
y_train = y_train + 1
y_test = y_test + 1;

T = Float32
X_train = Array{T}((X_train - minimum(X_train))/(maximum(X_train) - minimum(X_train)))
y_train = Array{Int64}(y_train)
X_test = Array{T}(X_test - minimum(X_test))/(maximum(X_test) - minimum(X_test)) 
y_test = Array{Int64}(y_test);

Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:64
 [2] Array(::Type{Float64}, ::Int64, ::Int64) at ./deprecated.jl:51
 [3] traindata() at /Users/macpro/.julia/v0.6/MNIST/src/MNIST.jl:88
 [4] include_string(::String, ::String) at ./loading.jl:515
 [5] execute_request(::ZMQ.Socket, ::IJulia.Msg) at /Users/macpro/.julia/v0.6/IJulia/src/execute_request.jl:156
 [6] eventloop(::ZMQ.Socket) at /Users/macpro/.julia/v0.6/IJulia/src/eventloop.jl:8
 [7] (::IJulia.##9#12)() at ./task.jl:335
while loading In[4], in expression starting on line 1
Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:64
 [2] Array(::Type{Float64}, ::Int64) at ./deprecated.jl:51
 [3] traindata() at /Users/macpro/.julia/v0.6/MNIST/src/MNIST.jl:89
 [4] include_string(::String, ::String) at ./loading.jl:515
 [5] execute_request(::ZMQ.Socket, ::IJulia.Msg) at /Users/macpro/.julia/v0.6/IJulia/src/execute_request.jl:156
 [6] eventloop(::ZMQ.Socket) at /Users/macpro/.julia/v0.6/IJulia/src/eventloop.

In [5]:
@benchmark MulticlassPerceptron1.fit!(percep1, X_train, y_train, 1, 0.0001)

BenchmarkTools.Trial: 
  memory estimate:  584.59 MiB
  allocs estimate:  655360
  --------------
  minimum time:     1.078 s (5.89% GC)
  median time:      1.086 s (5.72% GC)
  mean time:        1.088 s (5.83% GC)
  maximum time:     1.102 s (6.76% GC)
  --------------
  samples:          5
  evals/sample:     1

#### MulticlassPerceptron2

- Using views instead of copying examples

In [6]:
@benchmark MulticlassPerceptron2.fit!(percep2, X_train, y_train, 1, 0.0001)

BenchmarkTools.Trial: 
  memory estimate:  196.29 MiB
  allocs estimate:  412175
  --------------
  minimum time:     744.382 ms (2.55% GC)
  median time:      750.102 ms (2.71% GC)
  mean time:        749.016 ms (2.83% GC)
  maximum time:     751.086 ms (3.09% GC)
  --------------
  samples:          7
  evals/sample:     1

#### MulticlassPerceptron3

- Using views instead of copying examples
- using inbounds


In [7]:
@benchmark MulticlassPerceptron3.fit!(percep3, X_train, y_train, 1, 0.0001)

BenchmarkTools.Trial: 
  memory estimate:  162.88 MiB
  allocs estimate:  170838
  --------------
  minimum time:     728.654 ms (2.33% GC)
  median time:      732.285 ms (2.71% GC)
  mean time:        732.387 ms (2.65% GC)
  maximum time:     736.384 ms (2.79% GC)
  --------------
  samples:          7
  evals/sample:     1

#### MulticlassPerceptron4

- Using views instead of copying examples
- using views
- prealocated vector for predicting all datapoints
- using .* sintax for loop fusion

In [8]:
@benchmark MulticlassPerceptron4.fit!(percep4, X_train, y_train, 1, 0.0001)

BenchmarkTools.Trial: 
  memory estimate:  60.98 MiB
  allocs estimate:  239088
  --------------
  minimum time:     701.295 ms (1.18% GC)
  median time:      704.028 ms (1.18% GC)
  mean time:        704.046 ms (1.14% GC)
  maximum time:     706.259 ms (1.26% GC)
  --------------
  samples:          8
  evals/sample:     1

#### MulticlassPerceptron5

**What else can be improved?**

**Can we push the code to memory estimate 0 ?**

**Are we really using the BLAS at the fullest potential?**
